In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset using EIN as index
charity_df = pd.read_csv("charity_data.csv", index_col="EIN")
charity_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Store the names of all charities
charity_names = charity_df['NAME']
charity_names_df = pd.DataFrame(data=charity_names, index=charity_df.index, columns=["NAME"])
charity_names_df.head()

,NAME
EIN,
10520599,BLUE KNIGHTS MOTORCYCLE CLUB
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS
10553066,SOUTHSIDE ATHLETIC ASSOCIATION
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT


In [3]:
# Remove Name column
charity_df.drop(columns=["NAME"], inplace=True)
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
EIN,,,,,,,,,,
10520599,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
10531628,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
10547893,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
10553066,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
10556103,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [5]:
#%%
# Check the unique value counts to see if binning is needed
classification_counts = charity_df.CLASSIFICATION.value_counts()
classification_counts.plot.density()

In [6]:
# Determine which values to replace
replace_classifications = list(classification_counts[classification_counts < 1000].index)

# Replace in DataFrame
for classification in replace_classifications:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classification, "Other")

# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Check the unique value counts to see if binning is needed
apptype_counts = charity_df.APPLICATION_TYPE.value_counts()
apptype_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Determine which values to replace
replace_apptypes = list(apptype_counts[apptype_counts < 700].index)

# Replace in DataFrame
for apptype in replace_apptypes:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(apptype, "Other")
    
# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]), index=charity_df.index)

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10531628,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10547893,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10553066,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10556103,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Drop Special_Considerations_Y because it is a binary variable
encode_df = encode_df.drop("SPECIAL_CONSIDERATIONS_Y",1)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10531628,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10547893,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10553066,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10556103,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
# Merge one-hot encoded features and drop originals
charity_df = charity_df.merge(encode_df, left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N
EIN,,,,,,,,,,,,,,,,,,,,,
10520599,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10531628,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10547893,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10553066,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10556103,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop("IS_SUCCESSFUL",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 336       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 377
Trainable params: 377
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 5s 184us/sample - loss: 0.6152 - accuracy: 0.6865
Epoch 2/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5627 - accuracy: 0.7304
Epoch 3/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5584 - accuracy: 0.7317
Epoch 4/100
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5562 - accuracy: 0.7317
Epoch 5/100
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5555 - accuracy: 0.7310
Epoch 6/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5546 - accuracy: 0.7311
Epoch 7/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5540 - accuracy: 0.7316s - loss: 0.5536 - 
Epoch 8/100
25724/25724 [==============================] - 3s 97us/sample - loss: 0.5535 - accuracy: 0.7317
Epoch 9/100
25724/25724 [==============================] - 2s 91us/sample - loss: 0.5531 - 

25724/25724 [==============================] - 3s 100us/sample - loss: 0.5463 - accuracy: 0.7349
Epoch 75/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5464 - accuracy: 0.7348
Epoch 76/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5465 - accuracy: 0.7346
Epoch 77/100
25724/25724 [==============================] - 3s 111us/sample - loss: 0.5462 - accuracy: 0.7346 - loss: 0.5461 - 
Epoch 78/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5463 - accuracy: 0.7350
Epoch 79/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5465 - accuracy: 0.7352 - loss: 0.5467 - 
Epoch 80/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5459 - accuracy: 0.7353 - loss: 0.5457 - accuracy: 0.
Epoch 81/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5462 - accuracy: 0.7350
Epoch 82/100
25724/25724 [==============================] - 3s 104u

### Optimization: Target predictive accuracy >= 75%

In [16]:
# Try an additional neuron in 2nd hidden layer
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

new_nn = tf.keras.models.Sequential()

# First hidden layer
new_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
new_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
new_nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
new_nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 336       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 387
Trainable params: 387
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
new_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = new_nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = new_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 4s 155us/sample - loss: 0.6033 - accuracy: 0.6981
Epoch 2/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5663 - accuracy: 0.7210
Epoch 3/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5610 - accuracy: 0.7228
Epoch 4/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5592 - accuracy: 0.7232
Epoch 5/100
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5572 - accuracy: 0.7252
Epoch 6/100
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5550 - accuracy: 0.7288
Epoch 7/100
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5541 - accuracy: 0.7295
Epoch 8/100
25724/25724 [==============================] - 3s 102us/sample - loss: 0.5534 - accuracy: 0.7286
Epoch 9/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5528 - accuracy: 0.7

25724/25724 [==============================] - 2s 96us/sample - loss: 0.5457 - accuracy: 0.7339
Epoch 76/100
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5453 - accuracy: 0.7353
Epoch 77/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5454 - accuracy: 0.7348
Epoch 78/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5458 - accuracy: 0.7346
Epoch 79/100
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5458 - accuracy: 0.7352
Epoch 80/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5454 - accuracy: 0.7355
Epoch 81/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5456 - accuracy: 0.7353
Epoch 82/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5455 - accuracy: 0.7346
Epoch 83/100
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5454 - accuracy: 0.7349
Epoch 84/100
25724/25724 [======

The additional neuron in the 2nd layer very slightly increased accuracy.

In [ ]:
# Try additional neurons in 1st hidden layer, 9 original inputs = 18 total neurons
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 14
hidden_nodes_layer2 = 4

third_nn = tf.keras.models.Sequential()

# First hidden layer
third_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
third_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
third_nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
third_nn.summary()

In [19]:
# Compile the model
third_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = third_nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = third_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 3s 133us/sample - loss: 0.6105 - accuracy: 0.6896
Epoch 2/100
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5648 - accuracy: 0.7244
Epoch 3/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5582 - accuracy: 0.7286
Epoch 4/100
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5560 - accuracy: 0.7297
Epoch 5/100
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5542 - accuracy: 0.7300
Epoch 6/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5535 - accuracy: 0.7300
Epoch 7/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5524 - accuracy: 0.7307
Epoch 8/100
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5521 - accuracy: 0.7307
Epoch 9/100
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5514 - accuracy: 0.7320
Epoc

25724/25724 [==============================] - 3s 100us/sample - loss: 0.5427 - accuracy: 0.7359
Epoch 76/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5426 - accuracy: 0.7358
Epoch 77/100
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5421 - accuracy: 0.7380
Epoch 78/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5424 - accuracy: 0.7365
Epoch 79/100
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5426 - accuracy: 0.7352
Epoch 80/100
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5425 - accuracy: 0.7366
Epoch 81/100
25724/25724 [==============================] - 3s 103us/sample - loss: 0.5425 - accuracy: 0.7362
Epoch 82/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5423 - accuracy: 0.7369
Epoch 83/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5424 - accuracy: 0.7366
Epoch 84/100
25724/25724

8 neurons in 1st layer and 5 neurons in 2nd layer still best. Changing activation function between hidden layers.

In [22]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

act_nn = tf.keras.models.Sequential()

# First hidden layer
act_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation=tf.nn.leaky_relu))

# Second hidden layer
act_nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
act_nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
act_nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 8)                 336       
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 6         
Total params: 387
Trainable params: 387
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
act_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = act_nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = act_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 3s 135us/sample - loss: 0.6133 - accuracy: 0.6905
Epoch 2/100
25724/25724 [==============================] - 2s 86us/sample - loss: 0.5656 - accuracy: 0.7269
Epoch 3/100
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5600 - accuracy: 0.7294
Epoch 4/100
25724/25724 [==============================] - 2s 90us/sample - loss: 0.5573 - accuracy: 0.7285
Epoch 5/100
25724/25724 [==============================] - 3s 104us/sample - loss: 0.5558 - accuracy: 0.7294
Epoch 6/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5547 - accuracy: 0.7309
Epoch 7/100
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5544 - accuracy: 0.7305
Epoch 8/100
25724/25724 [==============================] - 2s 90us/sample - loss: 0.5534 - accuracy: 0.7309
Epoch 9/100
25724/25724 [==============================] - 2s 91us/sample - loss: 0.5531 - accuracy: 0.7307
Ep

25724/25724 [==============================] - 3s 108us/sample - loss: 0.5458 - accuracy: 0.7340
Epoch 76/100
25724/25724 [==============================] - 3s 108us/sample - loss: 0.5460 - accuracy: 0.7334 - l
Epoch 77/100
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5461 - accuracy: 0.7335
Epoch 78/100
25724/25724 [==============================] - 3s 102us/sample - loss: 0.5458 - accuracy: 0.7338 - loss: 0.544
Epoch 79/100
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5462 - accuracy: 0.7335
Epoch 80/100
25724/25724 [==============================] - 3s 117us/sample - loss: 0.5461 - accuracy: 0.7339
Epoch 81/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5457 - accuracy: 0.7348
Epoch 82/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5461 - accuracy: 0.7343
Epoch 83/100
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5458 - accuracy: 0.7338
Epoch

Accuracy went down slightly with different activation functions. Next step is to try removing features from data.